<a href="https://colab.research.google.com/github/MandavaNagasreeKalpana/Generative_Ai_analytics_vidya/blob/main/NYC_Taxi_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [31]:
data=pd.read_csv('/content/drive/MyDrive/nyc_taxi_trip_duration.csv')
data.info(),data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  729322 non-null  object 
 1   vendor_id           729322 non-null  int64  
 2   pickup_datetime     729322 non-null  object 
 3   dropoff_datetime    729322 non-null  object 
 4   passenger_count     729322 non-null  int64  
 5   pickup_longitude    729322 non-null  float64
 6   pickup_latitude     729322 non-null  float64
 7   dropoff_longitude   729322 non-null  float64
 8   dropoff_latitude    729322 non-null  float64
 9   store_and_fwd_flag  729322 non-null  object 
 10  trip_duration       729322 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 61.2+ MB


(None,
 id                    0
 vendor_id             0
 pickup_datetime       0
 dropoff_datetime      0
 passenger_count       0
 pickup_longitude      0
 pickup_latitude       0
 dropoff_longitude     0
 dropoff_latitude      0
 store_and_fwd_flag    0
 trip_duration         0
 dtype: int64)

In [32]:
import pandas as pd

data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])

data['pickup_year'] = data['pickup_datetime'].dt.year
data['pickup_month'] = data['pickup_datetime'].dt.month
data['pickup_day'] = data['pickup_datetime'].dt.day
data['pickup_day_of_week'] = data['pickup_datetime'].dt.dayofweek
data['pickup_hour'] = data['pickup_datetime'].dt.hour

data['calculated_trip_duration_minutes'] = (data['dropoff_datetime'] - data['pickup_datetime']).dt.total_seconds() / 60

data['original_trip_duration_minutes'] = data['trip_duration'] / 60

comparison = data[['calculated_trip_duration_minutes', 'original_trip_duration_minutes']].head()

display(data[['pickup_datetime','dropoff_datetime','pickup_year','pickup_month','pickup_day','pickup_day_of_week','pickup_hour','calculated_trip_duration_minutes','original_trip_duration_minutes']].head())
display(comparison)


,pickup_datetime,dropoff_datetime,pickup_year,pickup_month,pickup_day,pickup_day_of_week,pickup_hour,calculated_trip_duration_minutes,original_trip_duration_minutes
0,2016-02-29 16:40:21,2016-02-29 16:47:01,2016,2,29,0,16,6.666667,6.666667
1,2016-03-11 23:35:37,2016-03-11 23:53:57,2016,3,11,4,23,18.333333,18.333333
2,2016-02-21 17:59:33,2016-02-21 18:26:48,2016,2,21,6,17,27.250000,27.250000
3,2016-01-05 09:44:31,2016-01-05 10:03:32,2016,1,5,1,9,19.016667,19.016667
4,2016-02-17 06:42:23,2016-02-17 06:56:31,2016,2,17,2,6,14.133333,14.133333


,calculated_trip_duration_minutes,original_trip_duration_minutes
0,6.666667,6.666667
1,18.333333,18.333333
2,27.250000,27.250000
3,19.016667,19.016667
4,14.133333,14.133333


In [59]:
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Apply the Haversine distance calculation to the DataFrame
data['haversine_distance'] = data.apply(lambda row: haversine_distance(
    row['pickup_latitude'],
    row['pickup_longitude'],
    row['dropoff_latitude'],
    row['dropoff_longitude']
), axis=1)

display(data[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'haversine_distance']].head())

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,haversine_distance
0,40.778873,-73.953918,40.771164,-73.963875,1.199073
1,40.731743,-73.988312,40.694931,-73.994751,4.129111
2,40.721458,-73.997314,40.774918,-73.948029,7.250753
3,40.759720,-73.961670,40.780628,-73.956779,2.361097
4,40.708469,-74.017120,40.740631,-73.988182,4.328534


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 19 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   id                                729322 non-null  object        
 1   vendor_id                         729322 non-null  int64         
 2   pickup_datetime                   729322 non-null  datetime64[ns]
 3   dropoff_datetime                  729322 non-null  datetime64[ns]
 4   passenger_count                   729322 non-null  int64         
 5   pickup_longitude                  729322 non-null  float64       
 6   pickup_latitude                   729322 non-null  float64       
 7   dropoff_longitude                 729322 non-null  float64       
 8   dropoff_latitude                  729322 non-null  float64       
 9   store_and_fwd_flag                729322 non-null  object        
 10  trip_duration                   

In [69]:
X=data.drop(['trip_duration', 'id', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'calculated_trip_duration_minutes', 'original_trip_duration_minutes'],axis=1)
y = np.log1p(data['trip_duration'])
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=42)
X.shape,y.shape,X_train.shape,X_test.shape,y_train.shape,y_test.shape

((729322, 12), (729322,), (510525, 12), (218797, 12), (510525,), (218797,))

In [70]:
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

scaler = MinMaxScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train[numerical_cols]),
                                 columns=numerical_cols,
                                 index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test[numerical_cols]),
                                columns=numerical_cols,
                                index=X_test.index)

display(X_train_scaled.head())
display(X_test_scaled.head())

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,haversine_distance
605509,1.0,0.111111,0.855612,0.327001,0.855942,0.638097,0.002673
654962,0.0,0.111111,0.855723,0.329987,0.856455,0.646844,0.005533
644544,1.0,0.111111,0.855481,0.327580,0.855668,0.635477,0.000902
315304,0.0,0.111111,0.855115,0.325948,0.855169,0.632695,0.000374
246683,1.0,0.222222,0.855402,0.326873,0.855649,0.635151,0.000971


,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,haversine_distance
469114,1.0,0.111111,0.855728,0.328997,0.855898,0.641067,0.001840
694852,1.0,0.555556,0.855743,0.328130,0.855712,0.636482,0.000596
696324,0.0,0.111111,0.856277,0.330245,0.857764,0.647057,0.007267
356496,0.0,0.111111,0.855624,0.329042,0.855876,0.638968,0.000957
645318,0.0,0.333333,0.856040,0.330835,0.855606,0.638449,0.003474


In [83]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_scaled, y_train)


LinearRegression()

In [84]:
y_pred = np.expm1(model.predict(X_test_scaled))
y_test_actual = np.expm1(y_test)
mse = mean_squared_error(y_test_actual, y_pred)
r2 = r2_score(y_test_actual, y_pred)

print("MSE:", mse)
print("R2:", r2)


MSE: 3.451487447551967e+43
R2: -1.2976533391938598e+36
